In [3]:
from pathlib import Path

import numpy as np
import pandas as pd

from calculate_preferences import calcPref

# Change to your data directory
basedir = Path('/Volumes/data-1/behavior/hakan')


### Load data

In [4]:
data_path = basedir/'Kucukdereli_atal_data/data_table_all_mice.h5'
data_df = pd.read_hdf(data_path, key='data')

data_path = basedir/'Kucukdereli_atal_data/raw_behavior_data_males.h5'
male_data = pd.read_hdf(data_path, key='data')

data_path = basedir/'Kucukdereli_atal_data/raw_behavior_data_females.h5'
female_data = pd.read_hdf(data_path, key='data')


In [5]:
# Additional imports and setting up the plots
from scipy.stats import pearsonr, linregress, ttest_rel, bartlett
from statsmodels.stats.multitest import multipletests

import matplotlib.pyplot as plt

font = {'family' : 'sans',
        'weight' : 'regular',
        'size'   : 8}
plt.rc('font', **font)
plt.rc('axes', linewidth=0.8)

colors = {'gray':'#D4D4D4', 'darkgray':'#545454', 
          'male':'#FF5E74', 'male_light':'#FFABB6', 
          'female':'#00B7FF', 'female_light':'#9EE3FF'}


In [35]:
stim = df.query(f'session in [1,2] & stim!=0 & {condition}==True').groupby('cueid').dt.sum()
neutral = df.query(f'session in [1,2] & neutral!=0 & {condition}==True').groupby('cueid').dt.sum()

In [38]:
neutral.sample(n=len(neutral), replace=True, random_state=42)

cueid
103    137.519797
459      4.111459
353     17.299079
272      0.401140
107    213.020579
          ...    
366      1.715142
174     16.265928
281     13.738514
114    266.596166
289     14.755450
Name: dt, Length: 487, dtype: float64

In [158]:
np.concatenate(df.to_numpy()).reshape(-1, 2)[:,0].mean()
# .query('mouse==@mouse & experiment==@experiment')

-0.3988265468305971

In [173]:
sex = 'female'
nshuffles = 1000

titles = {'preference':'Baseline', 'test':'AgRP only', 'test_':'AgRP + stress'}

pref_df = female_data.query('session in [1,2]').groupby(['mouse','experiment','day']).apply(calcPref, 'dwell')
shuffle_df = female_data.query('session in [1,2]').groupby(['mouse','experiment','day']).apply(calcPrefShuffle, nshuffles, 'dwell')
mice = shuffle_df.index.get_level_values('mouse').unique().to_numpy()
experiments = shuffle_df.index.get_level_values('experiment').unique().to_numpy()

fig, axs = plt.subplots(len(mice),len(experiments), figsize=(len(experiments)*1.6,len(mice)*1.2), dpi=150,
                        sharex=False, sharey=True, gridspec_kw={'wspace':0.2, 'hspace':1.5, 'width_ratios':[1,5,5]})

for (mouse, experiment), df in shuffle_df.apply(np.quantile, q=[.025, .975]).groupby(['mouse','experiment']):
    days = df.index.get_level_values('day').unique().to_numpy()
    
    i, j = np.argwhere(mouse==mice).flatten()[0], np.argwhere(experiment==experiments).flatten()[0]
    ax = axs[i,j]

    ax.set_ylim(-1,1)

    if experiment=='preference':
        ax.set_title(f'(Mouse {mouse})\n{titles[experiment]}')

        ax.set_ylabel('AgRP preference')

        ax.set_xlim([-0.5, 1.5])
        ax.set_xticks([1])
        ax.set_xticklabels([1])

        ax.plot([-0.5, 1.5], [0,0], color='k', lw=1, ls=(2,(2,2)), zorder=-1)
        ax.plot(1, pref_df.loc[mouse, experiment].mean(), color=colors[sex], marker='_', lw=1.2, clip_on=False, zorder=2)
        ax.plot([1,1], [np.concatenate(df.to_numpy()).reshape(-1, 2)[:,0].mean(), 
                        np.concatenate(df.to_numpy()).reshape(-1, 2)[:,1].mean()], 
                        color=colors[sex], lw=1.5, alpha=0.35, clip_on=False, zorder=2)
    else:
        ax.set_title(f'{titles[experiment]}')

        ax.set_xlim(days[0]-0.5, days[-1]+0.5)
        ax.set_xticks(days)
        ax.set_xticklabels(days)
        ax.set_xlabel('Test days')

        ax.plot([days[0]-0.5, days[-1]+0.5], [0,0], color='k', lw=1, ls=(2,(2,2)), zorder=-1)
        ax.fill_between(days, np.concatenate(df.to_numpy()).reshape(-1, 2)[:,0], 
                        np.concatenate(df.to_numpy()).reshape(-1, 2)[:,1],
                        color=colors[sex], alpha=0.35, lw=0, clip_on=False, zorder=1)
        ax.plot(days, pref_df.loc[mouse, experiment].to_numpy(), color=colors[sex], lw=1.2, clip_on=False, zorder=2)



In [ ]:
fig, axs = plt.subplots(len(table.mouse.unique()),1, figsize=(2,len(table.mouse.unique())*1.2), dpi=100)
fig.subplots_adjust(hspace=1)
axs = axs.ravel()

pref_df, pref_dist_df = [], []

c = 0
for (mouse, experiment), df in tqdm(table.query("experiment in ['preference'] & day in [1,2]").groupby(['mouse','experiment'])):
    if len(df):
        trial_n1 = df.query("day==1").query("trial_1!=0").trial.unique()
        stim_n1 = df.query("day==1").query('stim==1 & trial_1!=0').trial.unique()
        neutral_n1 = df.query("day==1").query('stim==0 & trial_1!=0').trial.unique()
        trial_n2 = df.query("day==2").query("trial_1!=0").trial.unique()
        stim_n2 = df.query("day==2").query('stim==1 & trial_1!=0').trial.unique()
        neutral_n2 = df.query("day==2").query('stim==0 & trial_1!=0').trial.unique()
        
        
        # Calculate shuffled preferences
        pref_dist = []
        for i in range(shuffle_n):
            stim_trials1 = np.random.choice(trial_n1, size=len(stim_n1), replace=False)
            neutral_trials1 = np.random.choice(trial_n1, size=len(neutral_n1), replace=False)
            stim_trials2 = np.random.choice(trial_n2, size=len(stim_n2), replace=False)
            neutral_trials2 = np.random.choice(trial_n2, size=len(neutral_n2), replace=False)
            
            stim_t1 = df.query("day==1 & trial in @stim_trials1 & dwell!=0 & trial_1!=0").dt.sum()
            neutral_t1 = df.query("day==1 & trial in @neutral_trials1 & dwell!=0 & trial_1!=0").dt.sum()
            stim_t2 = df.query("day==2 & trial in @stim_trials2 & dwell!=0 & trial_1!=0").dt.sum()
            neutral_t2 = df.query("day==2 & trial in @neutral_trials2 & dwell!=0 & trial_1!=0").dt.sum()
            stim_t = stim_t1 + stim_t2
            neutral_t = neutral_t1 + neutral_t2
            
            pref = (stim_t - neutral_t) / (stim_t + neutral_t)
            pref_dist.append(pref)
        ci_lo, ci_hi = np.quantile(pref_dist, [.025, .975])
        pref_dist_df.append(pd.DataFrame({'mouse':mouse,'date':df.date.unique()[0],
                                         'experiment':experiment,'day':1,
                                         'pref':np.sort(pref_dist)}))
        
        stim_t1 = df.query("day==1 & stim==1 & dwell!=0 & trial_1!=0").dt.sum()
        neutral_t1 = df.query("day==1 & stim==0 & dwell!=0 & trial_1!=0").dt.sum()
        stim_t2 = df.query("day==2 & stim==1 & dwell!=0 & trial_1!=0").dt.sum()
        neutral_t2 = df.query("day==2 & stim==0 & dwell!=0 & trial_1!=0").dt.sum()
        stim_t = stim_t1 + stim_t2
        neutral_t = neutral_t1 + neutral_t2
            
        pref = (stim_t - neutral_t) / (stim_t + neutral_t)
        pref_df.append(pd.DataFrame({'mouse':mouse,'date':df.date.unique()[0],
                                     'experiment':experiment,'day':1,
                                     'pref':pref}, index=[0]))
        
        ax = axs[c]
        ax.hist(pref_dist, color='lightgreen');
        if (pref>ci_hi or pref<ci_lo):
            ax.scatter(pref, 0, s=30, c='tomato', clip_on=False, zorder=99)
        else:
            ax.scatter(pref, 0, s=30, c='xkcd:purple blue', clip_on=False, zorder=99)
        ax.set_title(f"{mouse}, {experiment}, {1}")
        ax.set_xlim([-1,1])
        
        ax.plot([ci_lo,ci_hi], [0,0], linewidth=3, clip_on=False)
    
    c = c + 1
    
pref_df = pd.concat(pref_df, ignore_index=True)
pref_dist_df = pd.concat(pref_dist_df, ignore_index=True)
pref_df_female_preference = pref_df
pref_dist_df_female_preference = pref_dist_df
